# 1.Introduction 

This report presents the modeling of heat transfer in a copper beam using the finite difference method and boundary conditions applied at 10 C° from the left end and 50 C° from the right end. Heat transfer is a fundamental phenomenon encountered in various engineering applications, and understanding the temperature distribution in solid structures is crucial for designing efficient thermal systems.

The heat equation is a partial differential equation that describes how temperature changes over time in a given domain. In one dimension, the heat equation can be expressed as:

$$
    \kappa \frac{\partial^2{u}}{\partial{x}^2} = \frac{\partial{u}}{\partial{t}}
$$

where u(x, t) represents the temperature distribution along the beam at position x and time t, and α is the thermal diffusivity of the material.

In our case, we are considering the steady-state or time-independent heat equation. This means that the temperature distribution does not change with time, and the equation simplifies to:

$$
    \kappa \frac{\partial^2{u}}{\partial{x}^2} = 0
$$

We seek to solve this equation numerically using the finite difference method, which discretises the domain into dx intervals. The boundary conditions specify the temperatures at the ends of the beam, with a fixed temperature of 10°C at 0 cm from the left and 50°C at 80 cm from the right.

By solving the heat equation with these boundary conditions, we can determine the temperature profile along the copper beam. 

In [1]:
using BoundaryValueDiffEq
using Plots
using SparseArrays
using LinearAlgebra
using BenchmarkTools

# 2.Source Fonction 


In this part, I model a single force acting on the beam. To do this, I will create a function that takes the form of a Gaussian function with a maximum value A representing the magnitude of the force, a width represented by the parameter σ, and located at position X0 on the beam.


$$ f(x) = A\exp{(-\frac{(x-x_0)^2}{\sigma^2})} $$

For the heat equation, this force is actually a heat peak of magnitude A, width σ and located at position X0 on the beam. In this example, the heat peak is located 60 cm along the beam, with a narrow width.




In [ ]:
function source_function(x)
    X0 = 60.0
    σ = 0.8
    A = 10.
    return  A .* exp.(.-((x .- X0).^2)./σ^2)
end

In [ ]:
function source(N)
#Parameter
    l = 80.
    dx = l/N
#Mesh
    X_vect = Vector(0:dx:l)
# Function source
    f = source_function(X_vect)
    return f
end

# 3.Finite difference modelling.

The 1D time-independent heat equation is a mathematical model used to describe the distribution of heat in a one-dimensional medium over time.
$$\frac{\partial^2{u}}{\partial{x}^2} = 0$$
Where u represents the temperature distribution along the x-axis. To solve this equation numerically, we discretize the domain into small intervals dx and approximate the second derivative using finite differences.
$$
    \frac{\kappa(U_{i+1} -2U_{i} + U{i-1})}{dx^2} = 0
$$
With the boudarys conditions :

$$
    U_0 = 10\\
    U_N = 50
$$
This leads to a system of linear equations Au = b, where A is a tridiagonal matrix that represents the coefficients of the finite difference scheme, x is the vector containing the unknown temperature values at each discretized point. 
$$
A = \frac{\kappa}{dx^2} \begin{bmatrix}
1 & 0 & 0 & \dots & 0 \\
-1 & 2 & -1 & \dots & 0 \\
0 & -1 & 2 & \dots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
0  & \ddots & \ddots  & -1 & 2 \\
0 & \dots & \dots & 0 & 1 \\
\end{bmatrix}
$$

The vector b is is containing the known boundary conditions, so for this example the vector will be : 
$$
    \mathbf{b} = \begin{bmatrix} 10 \\ 0 \\ 0 \\ \vdots \\ 0 \\ 50\end{bmatrix}
$$

The heat source must be added to the linear system to get the real case. 

$$
    \mathbf{S} = \begin{bmatrix} f(0) \\ \vdots \\ \vdots \\ \vdots \\ \vdots \\ f(L)\end{bmatrix}
$$

So the linear system that we trying to solve is :

$$
   \fbox{ Au = b+S}
$$



Firstly, we will create the matrix A with the data used for a copper beam. We'll use the material density d, thermal conductivity tc and specifc heatsh. This will give : 
$$
    \kappa = \frac{tc}{sh \times d}
$$


In [ ]:
function matrix(n) #8 times faster than before (calculate with @btime)

# Parameter for a copper bar 
    d= 8.92 #density 
    sh= 0.092 #specific_heat
    tc= 0.95 #thermal_conductivity
    l= 80 #length
    c = ( tc / (sh * d))^(1/2)
    dx = l/ n
    k = 1 #normally it's k = c²
    
# Build of the tridiagonal Matrix
    A = spdiagm(-1 =>  -(k / dx^2)* ones(n), 0 => 2 * (k / dx^2) * ones(n+1), 1 => - (k / dx^2) * ones(n)) 
    A[1,1] = 1
    A[1,2] = 0
    A[n+1,n+1] = 1 
    A[n+1,n] = 0
    return A
end

In [ ]:
function heat(N)

#Parameter of the bar 
    l= 80 #length
    dx = l/ N  

# Build of the matrix A
    A = matrix(N)
    #print(A)

# Vector B from Au = B (Boundary)
    b = zeros(N+1)
    b[1] = 10
    b[end] = 50

# Vector S for the source power
    X_vect = Vector(0:dx:l)
    S = source_function(X_vect) 
    S[1] = 0                        #Not sure about this 
    S[end] = 0                      #Not sure about this 
                                    #Dont know if it's B[1] = 10 or S[1] + B[1] = U[1] = 50 same for n
    
# Solving of the Linear System
    u = A \ (b+S)   
    return u 
end 

We also need to test the calculation time for this resolution. We can see that for a 1000x1000 matrix, the resolution time is fairly short.

In [ ]:
@btime heat(1000)

# 4.Using BoundaryValueDiffEq from julia

Modeling with BoundaryValueDiffEq involves solving boundary value problems (BVPs) for differential equations in Julia. The BoundaryValueDiffEq package provides a comprehensive set of tools for solving BVPs, including a wide range of solvers and utilities.

To use BoundaryValueDiffEq, we define our BVP by specifying the differential equation, boundary conditions, and the domain or spatial range. The differential equation is typically defined as a function that calculates the derivatives of the dependent variables with respect to the independent variable(s). The boundary conditions define the values or relationships at the boundaries of the domain.

Once the BVP is defined, we create a BVProblem object, which encapsulates the differential equation, boundary conditions, and domain information. We can then use the solve() function to solve the BVP and obtain the solution.

BoundaryValueDiffEq supports various numerical methods for solving BVPs, including shooting methods, finite difference methods, and collocation methods. The choice of solver depends on the specific problem characteristics and requirements.

I will divide the programme into three parts : 

The first of which will explain the problem of diffusion. The problem can be explained in the same way as the heat equation.


   $$ \frac{\partial^2{u}}{\partial{x}^2} = - f $$
    $$ \frac{\partial{u}}{\partial{x}} = U_2  = flux $$
    $$ \frac{\partial^2{u}}{\partial{x}^2} = 0 - f $$


The second part is the boundarys are explicited. Indeed, there are two boudarys conditions.

$$
    U(0) = 10 $$
   $$ U(L) = 50
$$

The last part is the resolution of the diffusion function and the boundarys function. As before, the parameters are those of an 80cm copper beam. With this programme, my result (sol1) is an N x 2 matrix, but to plot the solution, I only need the first term of each line. So I create a loop to obtain the solution vector sol2.



In [ ]:
# Define diffusion equation using BoudaryValuDiffEq

In [ ]:
function diffusion!(du, u, p, x)
    # u[1]: concentration 
    # u[2]: fluc 
    # d/dx(concentration)  = flux 
    du[1] = u[2]
    # d/dx(flux)  = sourceFct
    du[2] = 0 - source_function(x)
end    

In [ ]:
# Define boundary conditions 

In [ ]:
function boundaries!(residual, u, p, x)
    # residual boundary condition in the left end-point 
    residual[1] = u[1][1] - 10
    # residual boundary condition in the left end-point
    residual[2] = u[end][1] - 50
end 

In [ ]:
# Set the boundary value problem to solve

In [ ]:
function set_boundarys(N)
    
# define spatial range 
    xspan = (0.0, 80.)
    
# Parameter for a copper bar 
    d= 8.92 #density 
    sh= 0.092 #specific_heat
    tc= 0.95 #thermal_conductivity
    l = 80.
    dx = l/N
    xvec = Vector(0:dx:l)

    c = ( tc / (sh * d))^(1/2) 
    p = [c^2]; 
    
# set the boundary value problem to solve
    bvp1 = BVProblem(diffusion!, boundaries!, [10., 50.], xspan,p)

# set the boundary value problem  
    sol1 = solve(bvp1, GeneralMIRK4(), dt = 0.8)
    
# Take the first terme of every sublist for the first solution 
    sol2 = zeros(length(sol1))
    for i in 1:length(sol1)
        sol2[i] = sol1[i][1]  
    end
    return sol2
end

We also need to test the calculation time for this resolution. We can see that for a 100 points, the resolution time is fairly short. But it is important to note that the finite difference mode is shorter.

In [ ]:
@btime set_boundarys(100)

# 5.Conclusion

In conclusion, the plots of the two methods are identical, and it can be seen that the source interferes with the solution of the heat equation, which is normal. However, using the finite difference method to solve the heat equation is better because of the reduced calculation time.
Moreover, the relative error between these two methods is around $2.10^{-3}$ maximum, which is another good point for the finite differential method.

In [ ]:
#Set the Parameter N 

In [ ]:
N = 100
l= 80.
dx = l/N

In [ ]:
# Plot the computed solution: the source solution 

In [ ]:
xvec = Vector(0:dx:l)
fvec = source_function.(xvec)

p1 = plot(xvec, fvec, label = "Source", xlabel = "x", ylabel = "Source")

p2 = plot(xvec, set_boundarys(N), label = "Reference", xlabel = "x", ylabel = "Reference_sol", linecolor = :red)

p3 = plot(xvec, heat(N), label = "Finite_Dif", xlabel = "x", ylabel = "Modele_sol", linecolor = :green)

plot(p1, p2, p3, layout = (3, 1))


In [ ]:
exact = set_boundarys(N)
modele = heat(N)
error = (abs.(exact .- modele) ./ exact) 
title = "Relative error for $N points"
plot(xvec, error, label = "Relative error", xlabel = "Length of the beam", ylabel = "Error", title = title)